In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\megap\AppData\Local\Temp\ipykernel_16700\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [7]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("C:/Users/megap/Desktop/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [11]:
print (data.columns)# Your code
print (data.head(3))

from sklearn.model_selection import train_test_split

x = data["text"]
y = data["label"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=32)


Index(['text', 'label'], dtype='object')
                                                text  label
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                           Will do.      0
2  Nora--Cheryl has emailed dozens of memos about...      0


## Data Preprocessing

In [15]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [18]:
import re
def clean(html):
    html = re.sub(r'<script.*?>.*?</script>', '',html, flags=re.DOTALL)
    html = re.sub(r'<style.*?>.*?</style>', '',html, flags=re.DOTALL)# Your code
    
    html = re.sub(r'<!--.*?-->','', html, flags=re.DOTALL)

    html = re.sub(r'<.*?>','', html)

    return html

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [22]:
def clean_text(text):
    text = re.sub(r'[^A-Za-z0-9 ]','',text)
    text = re.sub(r'\d+', '', text)# Your code
    text = re.sub(r'\b\w\b', '', text)
    text = re.sub(r'^\b\w\b', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'^\b[bB]', '', text)
    text = text.lower()

    return text.strip()

## Now let's work on removing stopwords
Remove the stopwords.

In [23]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_tfidf = tfidf_vectorizer.fit_transform(data['text'])
# Your code

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
     words = word_tokenize(text)
     lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
     return " ".join(lemmatized_words)
data['text_lemmatized'] = data['text'].apply(lemmatize_text)# Your code

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\megap\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\megap\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\megap\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\megap/nltk_data'
    - 'c:\\Users\\megap\\anaconda3\\nltk_data'
    - 'c:\\Users\\megap\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\megap\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\megap\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [33]:
ham_messages = data[data['label'] == 0]['text_lemmatized']
spam_messages = data[data['label'] == 1]['text_lemmatized']
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')

ham_counts = vectorizer.fit_transform(ham_messages)
spam_counts = vectorizer.fit_transform(spam_messages)

ham_word_counts = ham_counts.sum(axis=0).A1
spam_word_counts = spam_counts.sum(axis=0).A1

words = vectorizer.get_feature_names_out()

ham_freq = dict(zip(words, ham_word_counts))
spam_freq = dict(zip(words, spam_word_counts))
ham_top = pd.Series(ham_freq).sort_values(ascending=False).head(10)
print("Top 10 palavras em mensagens HAM:")
print(ham_top)
spam_top = pd.Series(spam_freq).sort_values(ascending=False).head(10)
print("\nTop 10 palavras em mensagens SPAM:")
print(spam_top)# Your code

KeyError: 'text_lemmatized'

## Extra features

In [ ]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code